In [1]:
!pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 6.3 MB/s eta 0:00:02
   ------ --------------------------------- 2.1/12.9 MB 5.9 MB/s eta 0:00:02
   ---------- ----------------------------- 3.4/12.9 MB 5.8 MB/s eta 0:00:02
   ------------ --------------------------- 3.9/12.9 MB 5.1 MB/s eta 0:00:02
   ------------ --------------------------- 3.9/12.9 MB 5.1 MB/s eta 0:00:02
   ------------- -------------------------- 4.2/12.9 MB 3.4 MB/s eta 0:00:03
   -------------- ------------------------- 4.7/12.9 MB 3.3 MB/s eta 0:00:03
   ----------------- ---------------------- 5.8/12.9 MB 3.7 MB/s eta 0:00:02
   ------------------- -------------------- 6.3/12.9 MB 3.5 MB/s eta 0:00:02
   --------------------- ------------------ 6.8/12.9 MB 3.5 MB/s eta 0:00:02
   ------

In [1]:
import cv2

face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
smile_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")
eye_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
glasses_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml")


In [2]:
video_capture = cv2.VideoCapture(0)

In [3]:
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))

    if len(faces) == 0:
        return vid
    
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 2)

    roi_gray = gray_image[y:y+h, x:x+w]
    roi_color = vid[y:y+h, x:x+w]
    
    smiles = smile_classifier.detectMultiScale(roi_gray, 1.8, 20)
    for(sx, sy, sw, sh) in smiles:
        cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (255, 0, 0), 1)
        cv2.putText(vid, 'Senyum', (x + sx, y + sy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
    eyes = eye_classifier.detectMultiScale(roi_gray)
    for(ex, ey, ew, eh) in eyes:
        cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 255), 1)
        cv2.putText(vid, 'Mata', (x + ex, y + ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
    glasses = glasses_classifier.detectMultiScale(roi_gray, 1.1, 5)
    for(gx, gy, gw, gh) in glasses:
        cv2.rectangle(roi_color, (gx, gy), (gx + gw, gy + gh), (0, 0, 255), 1)
        cv2.putText(vid, 'Kacamata', (x + gx, y + gy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    return vid

In [ ]:
while True:

    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully

    video_frame = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame

    cv2.imshow(
        "Peningkatan dari Face Detection", video_frame
    )  # display the processed frame in a window named "My Face Detection Project"

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()